In [2]:
import os
import json
import ftplib
import datetime as dt

with open("credentials/passwords.json", "r") as file:
    passwords = json.load(file)

class InvalidPathError(Exception):
    pass

class Bluehost:
    def __init__(self):
        self.ftp = None

    def __enter__(self):
        self.ftp = self.login()
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        try:
            if self.ftp:
                self.ftp.quit()
        finally:
            self.ftp = None
        return False

    def login(self):
        ftp_server = 'ftp.luh.sgg.mybluehost.me'
        ftp_username = 'mpuche2@luh.sgg.mybluehost.me'
        ftp_password = passwords["BLUEHOST"]
        self.ftp = ftplib.FTP(ftp_server)
        try:
            self.ftp.login(ftp_username, ftp_password)
            welcome_message = self.ftp.getwelcome()
            if welcome_message:
                print("===")
                print("FTP connection established successfully!")
                print("Welcome message:", welcome_message)
                print("===")
            else:
                print("FTP connection failed. Check your credentials.")
        except ftplib.all_errors as e:
            print("FTP connection failed:", e)
            self.ftp.quit()
        finally:
            return self.ftp

    def logout(self):
        if hasattr(self, 'ftp'):
            self.ftp.quit() 

    def file_exists(self, path):
        try:
            self.ftp.size(path)
            return True
        except ftplib.error_perm:
            return False

    def dir_exists(self, path):
        try:
            self.ftp.nlst(path)
            return True
        except ftplib.error_perm:
            return False

    def listdir_recursive_only_files(self, path = os.path.join("")):
        try:
            listdir = self.ftp.nlst(path)
        except Exception as e:
            print(f"Error listing dir {path}: {e}")
            return []

        cleaned_listdir = []
        for x in listdir:
            if x[0] == "." or x[-1] == ".":
                continue
            elif x.count('.') > 0:
                cleaned_listdir.append(x)
            else:
                cleaned_listdir.extend(self.listdir_recursive_only_files(x))

        cleaned_listdir = sorted(cleaned_listdir)
        return cleaned_listdir

    def listdir(self, path = os.path.join("")):
        try:
            listdir = self.ftp.nlst(path)
        except Exception as e:
            print(f"Error listing dir {path}: {e}")
            return []

        cleaned_listdir = []
        for x in listdir:
            if x == "." or x[-1] == "." or x == "cgi-bin":
                continue
            cleaned_listdir.append(x)

        cleaned_listdir = sorted(cleaned_listdir)
        return cleaned_listdir

    def upload_file(self, local_path, remote_path, overwrite=False):
        local_dir = os.path.dirname(local_path)
        remote_dir = os.path.dirname(remote_path)
        local_dir = local_dir if local_dir != "" else os.path.join(".")
        remote_dir = remote_dir if local_dir != "" else os.path.join(".")
        if not os.path.exists(local_dir):
            raise InvalidPathError(f"Local path dir doesn't exist: {local_dir}")
        if not os.path.exists(local_path):
            raise InvalidPathError(f"Local path doesn't exist: {local_path}")
        if not self.dir_exists(remote_dir):
            raise InvalidPathError(f"Remote dir doesn't exist: {remote_dir}")
        if not overwrite and self.file_exists(remote_path):
            raise InvalidPathError(f"Remote file already exists: {remote_path}")
        with open(local_path, 'rb') as file:
            response = self.ftp.storbinary(f"STOR {remote_path}", file)
        print("---")
        print(f"filename: {os.path.basename(local_path)}")
        print(f"local path: {local_path}")
        print(f"remote path: {remote_path}")
        print(f"response: {response}")

    def download_file(self, remote_path, local_path, overwrite=False):
        local_dir = os.path.dirname(local_path)
        remote_dir = os.path.dirname(remote_path)
        local_dir = local_dir if local_dir != "" else os.path.join(".")
        remote_dir = remote_dir if local_dir != "" else os.path.join(".")
        if not os.path.exists(local_dir):
            raise InvalidPathError(f"Local dir doesn't exist: {local_dir}")
        if not self.dir_exists(remote_dir):
            raise InvalidPathError(f"Remote dir doesn't exist: {remote_dir}")
        if not self.file_exists(remote_path):
            raise InvalidPathError(f"Remote file doesn't exists: {remote_path}")
        if os.path.exists(local_path) and overwrite == False:
            raise InvalidPathError(f"Local file already exist: {local_path}")
        if os.path.exists(local_path) and overwrite == "rename":
            os.rename(local_path, local_path + "_OLD_" + dt.datetime.now().strftime("_%Y%m%d%H%M%S"))
        with open(local_path, 'wb') as file:
            response = self.ftp.retrbinary('RETR ' + remote_path, file.write)
        print("---")
        print(f"filename: {os.path.basename(local_path)}")
        print(f"local path: {local_path}")
        print(f"remote path: {remote_path}")
        print(f"response: {response}")

In [3]:
# ftp_server = 'ftp.luh.sgg.mybluehost.me'
# # ftp_username = 'mpuche2@luh.sgg.mybluehost.me'
# # ftp_password = passwords["BLUEHOST"]
# ftp = ftplib.FTP(ftp_server)


In [7]:
# os has already been imported
# Bluehost() has been defined above (not shown here)
# Bluehost has and ftp property (which works as standard ftp)

def update_website_dev_in_bluehost():
    filenames = ["index.html", "script.js", "style.css"]
    with Bluehost() as bh:
        for filename in filenames:
            bh.upload_file(
                remote_path = os.path.join(".", "vocab", filename),
                local_path = os.path.join(".", filename),
                overwrite=True
            )
        contents = bh.ftp.dir("./vocab")
    print(contents)

def upload_questions():
    filename = "calculus.json"
    dir_path_local = os.path.join(".", "data", "questions")         
    dir_path_remote = os.path.join(".", "vocab", "data", "questions")
    file_path_local = os.path.join(dir_path_local, filename)
    file_path_remote = os.path.join(dir_path_remote, filename)
    assert os.path.exists(file_path_local)
    with Bluehost() as bh:
        bh.upload_file(file_path_local, file_path_remote , overwrite = True)
        contents = bh.ftp.dir(dir_path_remote)
    print(contents)

# update_website_dev_in_bluehost()
# upload_questions()

# !git add .
# !git commit -m message
# !git push
# !git status

In [4]:
def generate_B001():
    path = os.path.join(".", "filters", "filters.txt")
    with open(path, "r") as file:
        filters = file.read()

    filters = filters.strip().split("\n")
    filters = [filter for filter in filters if "[o] B001" in filter]
    filters = [filter[4:4+8] for filter in filters]

    path = os.path.join(".", "transcriptions", "books", "B001", "B001_TRANS_ALL.txt")
    with open(path, "r") as file:
        lines = file.read().strip().split("\n")

    book = ""
    include = True
    for line in lines:
        if line.startswith("B001"):
            if line[0: 8] in filters:
                include = True
            else:
                include =False
        if include == True:
            book += line + "\n"

    with open(os.path.join(".", "B001.txt"), "w") as file:
        file.write(book)

In [5]:
# with Bluehost() as bh:
#     bh.ftp.mkd("./vocab/data/questions")
#     print(bh.listdir("./vocab/data"))

In [12]:
with Bluehost() as bh:
    content = bh.listdir()
    print(content)

===
FTP connection established successfully!
Welcome message: 220---------- Welcome to Pure-FTPd [privsep] [TLS] ----------
220-You are user number 2 of 150 allowed.
220-Local time is now 04:06. Server port: 21.
220-IPv6 connections are also welcome on this server.
220 You will be disconnected after 15 minutes of inactivity.
===
['.ftpquota', '.htaccess', '.htaccess.no_htaccess.phpupgrader', '.well-known', 'B001_TEXTS_ALL.txt', 'B002_TEXTS_ALL.txt', 'B003_TEXTS_ALL.txt', 'B009_TEXTS_ALL.txt', 'all.html', 'audio', 'filters', 'index.html', 'index_dev.html', 'reader', 'script.js', 'script_fast.js', 'script_slow.js', 'script_slow_dev.js', 'style.css', 'tester', 'text', 'transcriptions', 'vocab']
